# 01 - Data Engineering: Ingesta y Limpieza

**Objetivo:** Descargar, limpiar y preparar los datos para análisis.

**Fuentes:**
- FRED API: CPI, Federal Funds Rate, Oil Price
- World Bank: Gold Price (1960-2025)

**Output:** `data/processed/dataset_modelo.csv`

## 1. Setup

In [10]:
# Librerías
import pandas as pd
import numpy as np
import requests
import openpyxl
import os
from dotenv import load_dotenv

# Directorio de trabajo
os.chdir("/Users/javiermondragon/Documents/data_projects/inflation-predictor")

# Cargar API key
load_dotenv()
api_key = os.getenv("FRED_API_KEY")
print(f"API Key cargada: {api_key[:5]}...")

# Crear carpetas si no existen
os.makedirs("data/raw", exist_ok=True)
os.makedirs("data/processed", exist_ok=True)
print("Setup completo")

API Key cargada: b1ab9...
Setup completo


## 2. Descarga de Datos

### 2.1 Series de FRED

In [2]:
# Definir series de FRED
series_fred = {
    "CPIAUCSL": "cpi",
    "FEDFUNDS": "fed_rate",
    "DCOILWTICO": "oil_price"
}

url = "https://api.stlouisfed.org/fred/series/observations"

# Verificar cada serie
print("Verificando series...\n")
for codigo, nombre in series_fred.items():
    params = {
        "series_id": codigo,
        "api_key": api_key,
        "file_type": "json"
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    if "observations" in data:
        print(f"✓ {codigo}: {len(data['observations'])} registros")
    else:
        print(f"✗ {codigo}: ERROR - {data.get('error_message')}")

Verificando series...

✓ CPIAUCSL: 949 registros
✓ FEDFUNDS: 859 registros
✓ DCOILWTICO: 10469 registros


In [3]:
# Descargar y guardar datos crudos
print("Descargando datos...\n")

for codigo, nombre in series_fred.items():
    params = {
        "series_id": codigo,
        "api_key": api_key,
        "file_type": "json"
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    df = pd.DataFrame(data["observations"])
    df.to_csv(f"data/raw/{nombre}_raw.csv", index=False)
    
    print(f"✓ {nombre}_raw.csv - {len(df)} filas")

print("\nDatos de FRED guardados")

Descargando datos...

✓ cpi_raw.csv - 949 filas
✓ fed_rate_raw.csv - 859 filas
✓ oil_price_raw.csv - 10469 filas

Datos de FRED guardados


### 2.2 Oro del World Bank

In [12]:
# Cargar oro del World Bank
# NOTA: Copiar CMO-Historical-Data-Monthly.xlsx a data/raw/

df_gold_wb = pd.read_excel('data/raw/CMO-Historical-Data-Monthly.xlsx', 
                           sheet_name='Monthly Prices', 
                           header=4)

# Procesar
df_gold_wb = df_gold_wb.rename(columns={'Unnamed: 0': 'date'})
df_gold_wb = df_gold_wb[['date', 'Gold']].copy()
df_gold_wb = df_gold_wb.iloc[1:]  # Quitar fila de unidades

# Convertir tipos
df_gold_wb['date'] = pd.to_datetime(df_gold_wb['date'].str.replace('M', '-'), format='%Y-%m')
df_gold_wb['Gold'] = pd.to_numeric(df_gold_wb['Gold'], errors='coerce')
df_gold_wb = df_gold_wb.rename(columns={'Gold': 'value'})

# Guardar
df_gold_wb.to_csv('data/raw/gold_price_raw.csv', index=False)

print(f"✓ gold_price_raw.csv - {len(df_gold_wb)} filas")
print(f"  Rango: {df_gold_wb['date'].min().strftime('%Y-%m')} a {df_gold_wb['date'].max().strftime('%Y-%m')}")

✓ gold_price_raw.csv - 792 filas
  Rango: 1960-01 a 2025-12


## 3. Perfilamiento de Datos

In [13]:
# Cargar todos los datasets
datasets = {
    'cpi': pd.read_csv('data/raw/cpi_raw.csv'),
    'fed_rate': pd.read_csv('data/raw/fed_rate_raw.csv'),
    'oil_price': pd.read_csv('data/raw/oil_price_raw.csv'),
    'gold_price': pd.read_csv('data/raw/gold_price_raw.csv')
}

# Perfil de cada dataset
print("PERFIL DE DATOS CRUDOS")
print("=" * 60)

for nombre, df in datasets.items():
    print(f"\n{nombre.upper()}")
    print(f"  Filas: {len(df)}")
    print(f"  Columnas: {list(df.columns)}")

PERFIL DE DATOS CRUDOS

CPI
  Filas: 949
  Columnas: ['realtime_start', 'realtime_end', 'date', 'value']

FED_RATE
  Filas: 859
  Columnas: ['realtime_start', 'realtime_end', 'date', 'value']

OIL_PRICE
  Filas: 10469
  Columnas: ['realtime_start', 'realtime_end', 'date', 'value']

GOLD_PRICE
  Filas: 792
  Columnas: ['date', 'value']


## 4. Identificar Problemas

In [14]:
# Buscar valores problemáticos en series de FRED
print("IDENTIFICACIÓN DE PROBLEMAS")
print("=" * 60)

archivos_fred = ['cpi', 'fed_rate', 'oil_price']

for nombre in archivos_fred:
    df = datasets[nombre]
    
    print(f"\n{nombre.upper()}")
    print(f"  Fechas: {df['date'].min()} a {df['date'].max()}")
    
    # Valores no numéricos
    numerico = pd.to_numeric(df['value'], errors='coerce')
    no_numericos = df[numerico.isna()]['value'].unique()
    print(f"  Valores no numéricos: {no_numericos if len(no_numericos) > 0 else 'Ninguno'}")
    print(f"  Cantidad: {len(df[numerico.isna()])}")

# Gold (ya procesado)
print(f"\nGOLD_PRICE")
df_gold = datasets['gold_price']
print(f"  Fechas: {df_gold['date'].min()} a {df_gold['date'].max()}")
print(f"  Nulos: {df_gold['value'].isna().sum()}")

IDENTIFICACIÓN DE PROBLEMAS

CPI
  Fechas: 1947-01-01 a 2026-01-01
  Valores no numéricos: ['.']
  Cantidad: 1

FED_RATE
  Fechas: 1954-07-01 a 2026-01-01
  Valores no numéricos: Ninguno
  Cantidad: 0

OIL_PRICE
  Fechas: 1986-01-02 a 2026-02-17
  Valores no numéricos: ['.']
  Cantidad: 369

GOLD_PRICE
  Fechas: 1960-01-01 a 2025-12-01
  Nulos: 0


## 5. Limpieza

In [15]:
def limpiar_serie_fred(ruta, nombre_columna):
    """
    Limpia una serie de FRED:
    - Selecciona columnas date y value
    - Reemplaza "." por NaN
    - Convierte tipos
    - Renombra columna value
    """
    df = pd.read_csv(ruta)
    df = df[['date', 'value']].copy()
    df['value'] = df['value'].replace('.', pd.NA)
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'])
    df = df.rename(columns={'value': nombre_columna})
    return df

def limpiar_serie_gold(ruta):
    """
    Limpia la serie de oro del World Bank.
    """
    df = pd.read_csv(ruta)
    df['date'] = pd.to_datetime(df['date'])
    df['value'] = pd.to_numeric(df['value'])
    df = df.rename(columns={'value': 'gold_price'})
    return df

In [17]:
# Limpiar cada serie
df_cpi = limpiar_serie_fred('data/raw/cpi_raw.csv', 'cpi')
df_fed = limpiar_serie_fred('data/raw/fed_rate_raw.csv', 'fed_rate')
df_oil = limpiar_serie_fred('data/raw/oil_price_raw.csv', 'oil_price')
df_gold = limpiar_serie_gold('data/raw/gold_price_raw.csv')

print("Series limpias:")
print(f"  CPI: {len(df_cpi)} filas, {df_cpi['cpi'].isna().sum()} nulos")
print(f"  FED: {len(df_fed)} filas, {df_fed['fed_rate'].isna().sum()} nulos")
print(f"  OIL: {len(df_oil)} filas, {df_oil['oil_price'].isna().sum()} nulos")
print(f"  GOLD: {len(df_gold)} filas, {df_gold['gold_price'].isna().sum()} nulos")

Series limpias:
  CPI: 949 filas, 1 nulos
  FED: 859 filas, 0 nulos
  OIL: 10469 filas, 369 nulos
  GOLD: 792 filas, 0 nulos


## 6. Unificar Frecuencias

In [18]:
# Ver frecuencias actuales
print("Frecuencias actuales:")
print(f"  CPI: Mensual ({df_cpi['date'].min().year}-{df_cpi['date'].max().year})")
print(f"  FED: Mensual ({df_fed['date'].min().year}-{df_fed['date'].max().year})")
print(f"  OIL: Diaria ({df_oil['date'].min().year}-{df_oil['date'].max().year})")
print(f"  GOLD: Mensual ({df_gold['date'].min().year}-{df_gold['date'].max().year})")

Frecuencias actuales:
  CPI: Mensual (1947-2026)
  FED: Mensual (1954-2026)
  OIL: Diaria (1986-2026)
  GOLD: Mensual (1960-2025)


In [19]:
# Convertir OIL de diario a mensual (promedio)
df_oil_mensual = df_oil.set_index('date').resample('MS').mean().reset_index()

print(f"OIL convertido: {len(df_oil)} diarios → {len(df_oil_mensual)} mensuales")

OIL convertido: 10469 diarios → 482 mensuales


## 7. Unir Datasets

In [20]:
# Unir todas las series
df_final = df_cpi.copy()
df_final = df_final.merge(df_fed, on='date', how='outer')
df_final = df_final.merge(df_oil_mensual, on='date', how='outer')
df_final = df_final.merge(df_gold, on='date', how='outer')

# Ordenar por fecha
df_final = df_final.sort_values('date').reset_index(drop=True)

print(f"Dataset unido: {len(df_final)} filas")
print(f"Rango: {df_final['date'].min()} a {df_final['date'].max()}")
print(f"\nNulos por columna:")
print(df_final.isna().sum())

Dataset unido: 950 filas
Rango: 1947-01-01 00:00:00 a 2026-02-01 00:00:00

Nulos por columna:
date            0
cpi             2
fed_rate       91
oil_price     468
gold_price    158
dtype: int64


In [25]:
# Filtrar datos completos
df_modelo = df_final.dropna().reset_index(drop=True)

print(f"Dataset final: {len(df_modelo)} filas con datos completos")
print(f"Desde: {df_modelo['date'].min()}")
print(f"Hasta: {df_modelo['date'].max()}")

Dataset final: 479 filas con datos completos
Desde: 1986-01-01 00:00:00
Hasta: 2025-12-01 00:00:00


## 8. Guardar Dataset Procesado

In [26]:
# Guardar
df_modelo.to_csv('data/processed/dataset_modelo.csv', index=False)

print("✓ Guardado: data/processed/dataset_modelo.csv")
print(f"\nResumen:")
print(f"  Filas: {len(df_modelo)}")
print(f"  Columnas: {list(df_modelo.columns)}")
print(f"  Período: {df_modelo['date'].min().strftime('%Y-%m')} a {df_modelo['date'].max().strftime('%Y-%m')}")

✓ Guardado: data/processed/dataset_modelo.csv

Resumen:
  Filas: 479
  Columnas: ['date', 'cpi', 'fed_rate', 'oil_price', 'gold_price']
  Período: 1986-01 a 2025-12


In [27]:
# Vista previa
df_modelo.head()

,date,cpi,fed_rate,oil_price,gold_price
0,1986-01-01,109.9,8.14,22.925455,347.48
1,1986-02-01,109.7,7.86,15.454737,338.89
2,1986-03-01,109.1,7.48,12.612500,345.70
3,1986-04-01,108.7,6.99,12.843636,340.44
4,1986-05-01,109.0,6.85,15.377619,342.40


In [28]:
df_modelo.tail()

,date,cpi,fed_rate,oil_price,gold_price
474,2025-07-01,322.169,4.33,68.390909,3340.15
475,2025-08-01,323.291,4.33,64.864286,3368.03
476,2025-09-01,324.245,4.22,63.959048,3667.68
477,2025-11-01,325.063,3.88,60.062222,4087.19
478,2025-12-01,326.031,3.72,57.972273,4309.23
